In [1]:
import json
import torch
import os
import random
import argparse
import logging
logger = logging.getLogger(__name__)

In [2]:
class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 input_tokens,
                 input_ids,
                 idx,
                 label,

    ):
        self.input_tokens = input_tokens
        self.input_ids = input_ids
        self.idx=str(idx)
        self.label=label

# TODO use similar code to convert code examples to features
def convert_examples_to_features(js,tokenizer):#,args):
    #source
    code=js['func']
    code_tokens=tokenizer.tokenize(code)[:512-2]
    source_tokens =[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
    #source_ids =  tokenizer.convert_tokens_to_ids(source_tokens)
    source_ids_rf = tokenizer.convert_tokens_to_ids(code_tokens)
    #padding_length = 512 - len(source_ids)
    #source_ids+=[tokenizer.pad_token_id]*padding_length
    padding_length = 512 - len(source_ids_rf)
    source_ids_rf+=[tokenizer.pad_token_id]*padding_length
    return InputFeatures(source_tokens,source_ids_rf,js['idx'],js['target'])

    logger.info("Loading %d examples from %s", len(data), file_path)

In [3]:
# Vectorize the code using CodeBert
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=2)
#encoder_model = encoder_model.to(device)


data_train = [] # append examples from json
file_path = "data/primevul_train.jsonl"
with open(file_path) as f:
  print("Test")
  for line in f:
    #print(line)
    line=line.strip()
    js=json.loads(line)
    code=js['func'] # code example
    data_train.append(js)

print(data_train[0]['target']) # label of first sample
print(data_train[0]['func']) # code of first sample, to be tokenized and converted into "numbers"


input_features = []
#parser = argparse.ArgumentParser()
#parser.add_argument("--block_size", default=512, type=int)
  
#args = parser.parse_args()
#print(args)

# data_train contains json objects
for code_example in data_train:
  input_features.append(convert_examples_to_features(js=code_example, tokenizer=tokenizer))#, args=args))

/Users/katharinastengg/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/katharinastengg/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Test


Token indices sequence length is longer than the specified maximum sequence length for this model (16860 > 512). Running this sequence through the model will result in indexing errors


1
 long ssl_get_algorithm2(SSL *s)
        {
        long alg2 = s->s3->tmp.new_cipher->algorithm2;
       if (TLS1_get_version(s) >= TLS1_2_VERSION &&
            alg2 == (SSL_HANDSHAKE_MAC_DEFAULT|TLS1_PRF))
                return SSL_HANDSHAKE_MAC_SHA256 | TLS1_PRF_SHA256;
        return alg2;
	}



In [4]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import StratifiedShuffleSplit
import xgboost

print(input_features[0].label)
print(input_features[0].input_ids)

X_train = [code_example.input_ids for code_example in input_features]
y_train = [code_example.label for code_example in input_features]

#smoteTomek = SMOTETomek()
#features_res, labels_res = smoteTomek.fit_resample(features, labels)

random_oversampler = RandomOverSampler()
features_res, labels_res = random_oversampler.fit_resample(X_train, y_train)

#random_forest_classifier = RandomForestClassifier()
#random_forest_classifier.fit(features_res, labels_res)
xgb_model = xgboost.XGBClassifier(objective="binary:logistic", random_state=42)

xgb_model.fit(X_train, y_train)

1
[251, 579, 9996, 1215, 6460, 1215, 337, 47549, 176, 1640, 46535, 1009, 29, 43, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 25522, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 251, 1076, 571, 176, 5457, 579, 46613, 29, 246, 46613, 48743, 4, 4651, 1215, 438, 38914, 46613, 337, 47549, 176, 131, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 114, 36, 565, 10463, 134, 1215, 6460, 1215, 21747, 1640, 29, 43, 49095, 46402, 134, 1215, 176, 1215, 48195, 48200, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1076, 571, 176, 45994, 36, 46535, 1215, 725, 1889, 5433, 6826, 6961, 1215, 41458, 1215, 10089, 47697, 15483, 565, 10463, 134, 1215, 4454, 597, 35122, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 671, 43257, 1215, 725, 1889, 5433, 6826, 6961, 1215, 41458, 1215, 45004, 24554, 1721, 46402, 134, 1215, 4454, 597, 1215, 45004, 24554, 131, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 671, 1076, 571, 176, 131, 501

/Users/katharinastengg/Library/Python/3.9/lib/python/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/katharinastengg/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/Users/katharinastengg/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [5]:
data_test = [] # append examples from json
file_path = "data/primevul_test.jsonl"
with open(file_path) as f:
  print("Test")
  for line in f:
    #print(line)
    line=line.strip()
    js=json.loads(line)
    code=js['func'] # code example
    data_test.append(js)

print(data_test[0]['target']) # label of first sample
print(data_test[0]['func']) # code of first sample, to be tokenized and converted into "numbers"


test_features = []
#parser = argparse.ArgumentParser()
#parser.add_argument("--block_size", default=512, type=int)
  
#args = parser.parse_args()
#print(args)

# data_train contains json objects
for code_example in data_test:
  test_features.append(convert_examples_to_features(js=code_example, tokenizer=tokenizer))#, args=args))

print(len(test_features))
# check if first InputFeature object in list is correct :)
print(test_features[0].label)
print(test_features[0].input_tokens)
print(test_features[0].input_ids)
print(test_features[0].idx)


X_test = [code_example.input_ids for code_example in test_features]
y_test = [code_example.label for code_example in test_features]

#y_pred = random_forest_classifier.predict(X_test)

y_pred = xgb_model.predict(X_test)

Test
0
std::string CellularNetwork::ActivationStateToString(
    ActivationState activation_state) {
  switch (activation_state) {
    case ACTIVATION_STATE_ACTIVATED:
      return l10n_util::GetStringUTF8(
          IDS_CHROMEOS_NETWORK_ACTIVATION_STATE_ACTIVATED);
      break;
    case ACTIVATION_STATE_ACTIVATING:
      return l10n_util::GetStringUTF8(
          IDS_CHROMEOS_NETWORK_ACTIVATION_STATE_ACTIVATING);
      break;
    case ACTIVATION_STATE_NOT_ACTIVATED:
      return l10n_util::GetStringUTF8(
          IDS_CHROMEOS_NETWORK_ACTIVATION_STATE_NOT_ACTIVATED);
      break;
    case ACTIVATION_STATE_PARTIALLY_ACTIVATED:
      return l10n_util::GetStringUTF8(
          IDS_CHROMEOS_NETWORK_ACTIVATION_STATE_PARTIALLY_ACTIVATED);
      break;
    default:
      return l10n_util::GetStringUTF8(
          IDS_CHROMEOS_NETWORK_ACTIVATION_STATE_UNKNOWN);
      break;
  }
}

24788
0
['<s>', 'std', '::', 'string', 'ĠCellular', 'Network', '::', 'Activ', 'ation', 'State', 'To', 'String', '

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, confusion_matrix, classification_report

print(y_pred)
print(classification_report(y_test, y_pred))

auc = roc_auc_score(y_true=y_test, y_score=y_pred)
mcc = matthews_corrcoef(y_true=y_test, y_pred=y_pred)
print("AUC: ", auc)
print("MCC: ",mcc)

[0 0 0 ... 0 0 0]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     24239
           1       0.50      0.04      0.07       549

    accuracy                           0.98     24788
   macro avg       0.74      0.52      0.53     24788
weighted avg       0.97      0.98      0.97     24788

AUC:  0.5186924968725602
MCC:  0.13377152694832536


In [7]:
# Retrain codeBert
#tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
#model = AutoModel.from_pretrained("microsoft/codebert-base")

# training data = X_train, y_train
# test data = X_test, y_test


     

# attention mask differentiates padding from no padding 

In [ ]:
from transformers import DataCollatorWithPadding
import pandas as pd 
import json 

# Pad to the longest sequence in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess_function(js):
   #code=js['func']
   return tokenizer(js['func'], truncation=True)

# Tokenize train/test data
#data_train_df = pd.DataFrame()
data_list = []
with open("data/primevul_train.jsonl", "r") as file:
   for line in file:
      data_line_json=line.strip()
      js=json.loads(data_line_json)
      #data_train_df = pd.concat([data_train_df, pd.DataFrame(data_line_json)])
      data_list.append(js)

data_train_df = pd.DataFrame(data_list)
data_train_df = data_list
data_list = []
data_test_df = pd.DataFrame()
with open("data/primevul_test.jsonl", "r") as file:
   for line in file:
      data_line_json=line.strip()
      js=json.loads(data_line_json)
      #data_test_df = pd.concat([data_test_df, pd.DataFrame(data_line_json)])
      data_list.append(js)
data_test_df = pd.DataFrame(data_list)
data_test_df = data_list
#print(data_train_df.head())
#print(data_test_df.head())


# Trainer requires data to be in format 'input_ids', 'attention_mask', 'labels'



class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 input_tokens,
               #  input_ids,
                # idx,
                 label,

    ):
        self.input_tokens = input_tokens
        #self.input_ids = input_ids
       # self.idx=str(idx)
        self.label=label

    def __getitem__(self, i):       
        return torch.tensor(self.examples[i].input_ids),torch.tensor(self.examples[i].label)

# TODO use similar code to convert code examples to features
def convert_examples_to_features(js,tokenizer):#,args):
    #source
    code=js['func']
    #code_tokens=tokenizer.tokenize(code)[:512-2]
    code_tokens = tokenizer(code, truncation=True)
    #source_tokens =[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
    #source_ids =  tokenizer.convert_tokens_to_ids(source_tokens)
    #source_ids_rf = tokenizer.convert_tokens_to_ids(code_tokens)
    #padding_length = 512 - len(source_ids)
    #source_ids+=[tokenizer.pad_token_id]*padding_length
    #padding_length = 512 - len(source_ids_rf)
    #source_ids_rf+=[tokenizer.pad_token_id]*padding_length
    #    return InputFeatures(source_tokens,source_ids_rf,js['idx'],js['target'])
    return code_tokens, torch.tensor(js['label'])
    #logger.info("Loading %d examples from %s", len(data), file_path)

tokenized_train = []
tokenized_test = []

# data_train contains json objects
for code_example in data_train_df:
  tokenized_train.append(convert_examples_to_features(js=code_example, tokenizer=tokenizer))#, args=args))
for code_example in data_test_df:
  tokenized_test.append(convert_examples_to_features(js=code_example, tokenizer=tokenizer))#, args=args))

KeyboardInterrupt: 

In [ ]:

from datasets import load_dataset, DatasetDict
dataset = load_dataset("yelp_review_full")

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=2)

#'text': blabla, 'this': blabla
print(dataset.shape)

def tokenize_function(json):
    return tokenizer(json["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'train': (650000, 2), 'test': (50000, 2)}


Map: 100%|██████████| 50000/50000 [00:07<00:00, 6981.32 examples/s]


In [ ]:
import json 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import DatasetDict, Dataset

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=2)


data_list = []
labels = []
with open("data/primevul_train.jsonl", "r") as file:
   for line in file:
      data_line_json=line.strip()
      js=json.loads(data_line_json)
      #data_train_df = pd.concat([data_train_df, pd.DataFrame(data_line_json)])
      data_list.append(js['func'])
      labels.append(js['target'])


ds = Dataset.from_dict({'functions':data_list, 
                        'labels':labels})
#data_dict = DatasetDict({
   #})


def tokenize_function(json):
    return tokenizer(json["functions"], padding="max_length", truncation=True)

#tokenized_datasets = data_dict.map(tokenize_function, batched=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: dictionary update sequence element #0 has length 175797; 2 is required

In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})


In [53]:
import numpy as np
import evaluate


def compute_metrics(eval_pred):
    """Calculate F1 score"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    load_f1 = evaluate.load("f1")
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"f1": f1}

In [54]:
from transformers import TrainingArguments, Trainer

# Training arguments for parameter tuning
training_args = TrainingArguments(
   "model",
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=1, ## currently only 1 epoch!
   weight_decay=0.01,
   save_strategy="epoch",
   report_to="none"
)

# Trainer which executes the training process
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   processing_class=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [11]:
#!pip3 install accelerate
#!pip3 install transformers[torch]

In [55]:
trainer.train()
trainer.evaluate(tokenized_test)

AttributeError: 'list' object has no attribute 'keys'